<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C6-Extending_Burp_Proxy/Using_Bing_For_Burp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Using Bing for Burp__*

It's not uncommon for a single web server to serve several web applications, some of which you might not be aware of. If you're attacking the server, you should do your best to discover these other hostnames, because they might give you an easier way to get a shell. It's not rare to find an insecure web application, or even development resources, located on the same machine as your target. Microsoft's __Bing__ search engine has search capabilities that allow you to query Bing for all websites it finds on a single IP address using the _"IP"_ search modifier. Bing will also tell you all of the subdomains of a given domain if you use the _"domain"_ search modifier.

Now, we could use a scraper to submit these queries to Bing and then get the HTML in the results, but that would be bad manners (and also violate most search engines' term of use). In order to stay out of trouble, we'll instead use the Bing API to submit these queries programmatically and parse the results ourselves. (Visit _http://www.microsoft.com/en-us/bing/apis/bing-web-search-api/_ to get set up with your own free Bing API key). Except for a context menu, we won't implement any fancy Burp GUI additions with this extension, we'll simply output the resutls into Burp each time we run a query, and any detected URLs to Burp's target scope will be added automatically.

Because we already walked you through how to read the Burp API documentation and translate it into Python, let's get right to the code. Crack open __bhp_bing.py__ and hammer out the following:

In [ ]:
from burp import IBurpExtender
from burp import IContextMenuFactory

from java.net import URL
from java.util import ArrayList
from javax.swing import JMenuItem
from thread import start_new_thread

import json
import socket
import urllib

API_KEY = "YOURKEY" #[1]
API_HOST = "api.cognitive.microsoft.com"

class BurpExtender(IBurpExtender, IContextMenuFactory): #[2]
    def registerExtenderCallbacks(self, callbacks):
        self._callbacks = callbacks
        self._helpers = callbacks.getHelpers()
        self.context = None

        # We set up our extension
        callbacks.setExtensionName("BHP Bing")
        callbacks.registerContextMenuFactory(self) #[3]

        return

    def createMenuItems(self, context_menu):
        self.context = context_menu
        menu_list = ArrayList()
        menu_list.add(JMenuItem("Send to Bing", actionPerformed=self.bing_menu)) #[4]
        return menu_list

This is the first bit of our Bing extension. Make sure you paste your Bing API key in place __[1]__. You're allowed 1000 free searches per month. We begin by defining a __BurpExtender__ class __[2]__ that implements the standard __IBurpExtender__ interface, and the __IContextMenuFactory__, which allows us to provide a context menu when a user right-clicks a requests in Burp. This menu will display a _"Send to Bing"_ selection. We register a menu handler __[3]__ that will determine which site the user clicked, enabling us to construct our Bing queries. Then we set up a __createMenuItem__ method, which will receive an __IContextMenuInvocation__ object and use it to determine which HTTP request the user selected. The last step is to render the menu item and handle the click event with the __bing_menu__ method __[4]__.

Now let's perform the Bing query, output the results, and add any discovered virtual hosts to Burp's target scope:

In [ ]:
class BurpExtender(IBurpExtender, IContextMenuFactory):
    def bing_menu(self, event):
        # Grab the details of what the user clicked
        http_traffic = self.context.getSelectedMessages() #[1]

        print("%d requests highlighted" % len(http_traffic))

        for traffic in http_traffic:
            http_service = traffic.getHttpService()
            host = http_service.getHost()

            print("User selected host: %s" % host)
            self.bing_search(host)

        return

    def bing_search(self, host):
        # Check if we have an IP or hostname
        try:
            is_ip = bool(socket.inet_aton(host)) #[2]
        except socket.error:
            is_ip = False

        if is_ip:
            ip_address = host
            domain = False
        else:
            ip_address = socket.gethostbyname(host)
            domain = True

        start_new_thread(self.bing_query, ("ip:%s" % ip_address,)) #[3]

        if domain:
            start_new_thread(self.bing_query, ("domain:%s" % host,)) #[4]

The __bing_menu__ method gets triggered when the user clicks the context menu item we defined. We retrieve the highlighted HTTP requests __[1]__. Then we retrieve the host portion of each request and send it to the __bing_search__ method for further processing. The __bing_search__ method first determines if the host portion is an IP address or a hostname __[2]__. We then query Bing for all virtual hosts that have the same IP address __[3]__ as the host. If our extension received a domain as well, then we do a secondary search for any subdomains that Bing may have indexed __[4]__.

Now let's install the plumbing we'll need in order to send the request to Bing and parse the results using Burp's HTTP API. Add the following code within the __BurpExtender__ class:

In [ ]:
class BurpExtender(IBurpExtender, IContextMenuFactory):
    def bing_query(self, bing_query_string):
        print("Performing Bing search: %s" % bing_query_string)
        http_request = "GET https://%s//bing//v7.0/search?" % API_HOST
        # Encode our query
        http_request += "q=%s HTTP/1.1\r\n" % urllib.quote(bing_query_string)
        http_request += "Host: %s\r\n" % API_HOST
        http_request += "Connection:close\r\n"
        http_request += "Ocp-Apim_Subscription-Key: %s\r\n" % API_KEY #[1]
        http_request += "User-Agent: Black Hat Python\r\n\r\n"

        json_body = self._callbacks.makeHttpRequest(
            API_HOST, 443, True, http_request).toString() #[2]
        json_body = json_body.split("\r\n\r\n", 1)[1] #[3]

        try:
            response = json.loads(json_body) #[4]
        except (TypeError, ValueError) as err:
            print("No results from Bing %s" % err)
        else:
            sites = list()
            if response.get("webPages"):
                sites = response["webPages"]["value"]
            if len(sites):
                for site in sites:
                    print('*' * 100) #[5]
                    print("Name: %s" % site["name"])
                    print("URL: %s" % site["url"])
                    print("Description: %s" % site["snippet"])
                    print('*' * 100)

                    java_url = URL(site["url"])
                    if not self._callbacks.isInScope(java_url): #[6]
                        print("Adding %s to Burp scope" % site["url"])
                        self._callbacks.includeInScope(java_url)
                    else:
                        print("Empty response from Bing.: %s" % bing_query_string)

        return